In [1]:
%run ./time_series_encoding.py

synthetic_lf().collect().write_parquet("synthetic.parquet")
lf = load_and_preprocess_data("synthetic.parquet")

lf = lf.select(
    [pl.col("index")]
    + [pl.col(c) for c in list(reversed(lf.columns)) if c.startswith("logit")]
)
lf.head().collect()

index,logit[MEAN_ENCODED_feature1_540],logit[MEAN_ENCODED_feature1_510],logit[MEAN_ENCODED_feature1_480],logit[MEAN_ENCODED_feature1_450],logit[MEAN_ENCODED_feature1_420],logit[MEAN_ENCODED_feature1_390],logit[MEAN_ENCODED_feature1_360],logit[MEAN_ENCODED_feature1_330],logit[MEAN_ENCODED_feature1_300],logit[MEAN_ENCODED_feature1_270],logit[MEAN_ENCODED_feature1_240],logit[MEAN_ENCODED_feature1_210],logit[MEAN_ENCODED_feature1_180],logit[MEAN_ENCODED_feature1_150],logit[MEAN_ENCODED_feature1_120],logit[MEAN_ENCODED_feature1_90],logit[MEAN_ENCODED_feature1_60],logit[MEAN_ENCODED_feature1_30]
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,3.660958,7.965675,9.965675,11.965675,13.965675,15.965675,17.965675,19.965675,21.965675,23.965675,25.965675,27.965675,29.965675,31.965675,33.965675,35.965675,37.965675,39.965675
1,1.313476,4.273492,6.273492,8.273492,10.273492,12.273492,14.273492,16.273492,18.273492,20.273492,22.273492,24.273492,26.273492,28.273492,30.273492,32.273492,34.273492,36.273492
2,4.868407,9.618858,11.618858,13.618858,15.618858,17.618858,19.618858,21.618858,23.618858,25.618858,27.618858,29.618858,31.618858,33.618858,35.618858,37.618858,39.618858,41.618858
3,5.501992,10.442557,12.442557,14.442557,16.442557,18.442557,20.442557,22.442557,24.442557,26.442557,28.442557,30.442557,32.442557,34.442557,36.442557,38.442557,40.442557,42.442557
4,0.191092,2.240056,4.240056,6.240056,8.240056,10.240056,12.240056,14.240056,16.240056,18.240056,20.240056,22.240056,24.240056,26.240056,28.240056,30.240056,32.240056,34.240056


In [2]:
from catboost import CatBoostRegressor

ts = TimeSeriesEncoding(lf, CatBoostRegressor(verbose=0))
ts.encode_feature().collect()

Averaged MSE for 1 lags: 0.0629
Averaged MSE for 2 lags: 0.2163
Averaged MSE for 3 lags: 0.1053
Averaged MSE for 4 lags: 0.0867
Averaged MSE for 5 lags: 0.0739
Averaged MSE for 6 lags: 0.0577
Averaged MSE for 7 lags: 0.0801
Averaged MSE for 8 lags: 0.0637
Averaged MSE for 9 lags: 0.0573
Averaged MSE for 10 lags: 0.0582
Averaged MSE for 11 lags: 0.0535
Averaged MSE for 12 lags: 0.0619


<LazyFrame [2 cols, {"index": Int64, "feature1_current": Float64}] at 0x7F946DE12B90>

In [3]:
ts.find_optimal_lags()

11

In [5]:
pl.concat(
    [
        ts.encode_feature(),
        lf.select(
            ["logit[MEAN_ENCODED_feature1_30]", "logit[MEAN_ENCODED_feature1_60]"]
        ),
    ],
    how="horizontal",
).collect()

index,feature1_current,logit[MEAN_ENCODED_feature1_30],logit[MEAN_ENCODED_feature1_60]
i64,f64,f64,f64
0,41.960077,39.965675,37.965675
1,38.262564,36.273492,34.273492
2,43.597497,41.618858,39.618858
3,44.447554,42.442557,40.442557
4,36.27281,34.240056,32.240056
…,…,…,…
495,44.447554,42.419702,40.419702
496,36.507787,34.497376,32.497376
497,40.052175,38.043689,36.043689


In [ ]:
lf.columns[1:]

In [ ]:
lf1 = recode_columns(lf)

lf1.head().collect()


1. starting at the lag 2, continuing for `n_prior` lags (so 2 to 2+n_prior inclusive)
2. use the lag 2 value as the target, and the remaining columns as features to predict the lag 2 value
3. using the lag 1 value as the target, and the remaining columns as features, apply the fitted model to the lag 2 to lag 2+n_prior columns to estimate the lag 1 value
4. calculate the MSE of the estimated lag 1 value and the actual lag 1 value 

5. repeat steps 3 and 4, starting at lag 3, fitting the model to the lag 3 to lag 3+n_prior columns, estimating the lag 2 value, calculating the MSE of the estimated lag 2 value and the actual lag 2 value, and so on
6. continue until you have n_folds MSE values
7. return the mean and sd of the MSE values

In [ ]:
get_columns(lf1, "feature1", 5, 2 - 1).collect()

In [ ]:
"""

1. starting at the lag 2, continuing for `n_prior` lags (so 2 to 2+n_prior inclusive)
2. use the lag 2 value as the target, and the remaining columns as features to predict the lag 2 value
3. using the lag 1 value as the target, and the remaining columns as features, apply the fitted model to the lag 2 to lag 2+n_prior columns to estimate the lag 1 value
4. calculate the MSE of the estimated lag 1 value and the actual lag 1 value 

5. repeat steps 3 and 4, starting at lag 3, fitting the model to the lag 3 to lag 3+n_prior columns, estimating the lag 2 value, calculating the MSE of the estimated lag 2 value and the actual lag 2 value, and so on
6. continue until you have n_folds MSE values
7. return the mean and sd of the MSE values
"""


def lagged_forecast(lf: pl.LazyFrame, feature: str, n_folds: int, n_prior: int):
    # for fold in range(1, n_folds + 1):
    fold = 1

    # train model on lag 2 to lag 2+n_prior columns
    y_col = get_columns(lf, feature, n_folds).columns[0]
    x_cols = get_columns(lf, feature, n_folds, fold + 1 - 1).columns[1:]
    print(f"y_cols: {y_col}")
    print(f"x_cols: {x_cols}")
    y = lf.select(y_col).collect().to_numpy().reshape(-1, 1)

    # X0 = lf.select(
    #     [
    #         get_columns(lf, feature, n_folds, fold + i - 1)
    #         for i in range(1, n_prior + 2)
    #     ]
    # ).collect()

    # estimate lag 1 value

    return y


lagged_forecast(lf1, "feature1", 5, 3)

In [ ]:
get_columns(lf1, "feature1", 5, i).columns

In [ ]:
xy = X_y_generator(lf1, "feature1", 5, 2)
count = 0
for x, y in xy:
    count += 1
    print(f"count: {count}")
    print(
        f"\n\n==\nx:\n==\n\n{x.reshape(-1, 1)[:5]}\n\n==\ny:\n==\n\n{y.reshape(-1, 1)[:5]}"
    )

In [ ]:
def extract_features_and_lags(lf: pl.LazyFrame) -> tuple:
    features = [c.split("_")[2] for c in lf.select(cs.starts_with("logit")).columns]

    lags = [
        int(c.split("_")[-1].replace("]", ""))
        for c in lf.select(cs.starts_with("logit")).columns
    ]

    lf = lf.select(
        [pl.col("index")]
        + [
            pl.col(c).alias(f"{f}_{int(l/30)}")
            for c, f, l in zip(
                lf.select(cs.starts_with("logit")).columns, features, lags
            )
        ]
    )
    return lf, features, lags


df, feats, lags = extract_features_and_lags(lf)
df.head().collect()